# Running llama.cpp with VDS Tunnel in Google Colab

## Настройка

1. В Colab добавьте секреты (значок ключа слева):
   - vds_host: IP вашего VDS
   - ssh_key: приватный SSH-ключ в base64 (конвертируйте командой `base64 -w0 ~/.ssh/id_rsa`)

2. На VDS:
   - Добавьте публичный ключ в `~/.ssh/authorized_keys`
   - Откройте порт 8000
   - Разрешите создание обратного туннеля в `/etc/ssh/sshd_config`: `GatewayPorts yes`

3. Используйте GPU runtime (Runtime -> Change runtime type -> GPU)

## 1. Установка llama-cpp-python с поддержкой CUDA

In [ ]:
# Установка llama-cpp-python с поддержкой CUDA
import os
os.environ['CMAKE_ARGS'] = '-DGGML_CUDA=ON'
!pip install llama-cpp-python[server]

## 2. Настройка окружения и импорт зависимостей

In [ ]:
from google.colab import userdata
import os
import subprocess
import threading
import time
import base64

# Проверка наличия всех необходимых секретов
required_secrets = ['vds_host', 'ssh_key']
missing_secrets = [secret for secret in required_secrets if not userdata.get(secret)]
if missing_secrets:
    raise ValueError(f"Missing required secrets: {', '.join(missing_secrets)}\n"
                    "Add them in Colab: Left sidebar -> Key icon -> Add new secret\n"
                    "Required secrets:\n"
                    "- vds_host: IP адрес VDS\n"
                    "- ssh_key: SSH приватный ключ в base64")

# Создаем директорию .ssh и сохраняем ключ
ssh_dir = os.path.expanduser('~/.ssh')
os.makedirs(ssh_dir, mode=0o700, exist_ok=True)

# Декодируем и сохраняем SSH ключ
key_data = base64.b64decode(userdata.get('ssh_key')).decode()
key_path = os.path.join(ssh_dir, 'tunnel_key')
with open(key_path, 'w') as f:
    f.write(key_data)
os.chmod(key_path, 0o600)

# Конфигурация из секретов
VDS_HOST = userdata.get('vds_host')
VDS_USER = 'tunnel'  # Фиксированный пользователь из скрипта установки
LOCAL_PORT = 8000  # Порт для llama.cpp сервера

## 3. Создание модели для загрузки

Загрузим модель Qwen с помощью HuggingFace Hub

In [ ]:
!pip install huggingface_hub

from huggingface_hub import hf_hub_download
model_path = hf_hub_download(repo_id="Qwen/Qwen2-0.5B-Instruct-GGUF", filename="*q8_0.gguf")

## 4. Запуск SSH туннеля и llama.cpp сервера

In [ ]:
def run_and_print_output(process):
    """Helper function to continuously read and print process output"""
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output:
            print(output.strip())

# Создание SSH туннеля
ssh_command = f"ssh -N -R 0.0.0.0:{LOCAL_PORT}:localhost:{LOCAL_PORT} -o StrictHostKeyChecking=no -i {key_path} {VDS_USER}@{VDS_HOST}"
ssh_process = subprocess.Popen(
    ssh_command.split(),
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True
)

# Запуск llama.cpp сервера
server_command = f"python -m llama_cpp.server --model {model_path} --host 0.0.0.0 --port {LOCAL_PORT} --n_gpu_layers 35 --n_ctx 2048"
server_process = subprocess.Popen(
    server_command.split(),
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True
)

server_thread = threading.Thread(target=run_and_print_output, args=(server_process,))
server_thread.daemon = True
server_thread.start()

# Даем серверу время на запуск
time.sleep(10)

print(f"\n=== Your llama.cpp server is available at ===")
print(f"http://{VDS_HOST}:{LOCAL_PORT}")
print("=====================================")

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("Shutting down...")
    server_process.terminate()
    ssh_process.terminate()